Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Vertex AI Search - REST API 호출로 검색

Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search와 Langchain 을 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.

* Vertex AI Search Manual references
    * https://cloud.google.com/enterprise-search?hl=ko
    * https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search


이 소스에서 지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

#라이브러리 설치
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [1]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [3]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.
* 아래 SEARCH_URL Vertex AI Search를 구성 후 Integration 메뉴항목에서 추출된 값입니다. 

In [12]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL = "gemini-1.5-flash"
#SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/servingConfigs/default_search:search"
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/interpreter-rag-ds_1716974785800/servingConfigs/default_search:search"

In [13]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

vertexai.init(project=PROJECT_ID, location=REGION)

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()


### Vertex AI Search 검색

아래 코드는 REST API를 통해서 Verext AI Search에 직접 검색 쿼리를 보내는 예제입니다.
검색에 활용되는 패러미터는 ContentSearchSpec 에 따릅니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec

특히 ExtractiveContentSpec은 검색 컨텐츠를 좀더 상세하게 얻을수 있는 설정입니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec.ExtractiveContentSpec

In [28]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{
            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }              
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text


#### Search 결과 파싱 로직
아래 로직은 검색된 결과를 파싱하는 로직입니다.

In [32]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']

            #print(chunk)
            # print(f"content {chunk['content']}")

            # print(f"documentMetadata {chunk['documentMetadata']}")
            # print(f"documentMetadata {chunk['documentMetadata']['title']}")
            # print(f"documentMetadata {chunk['documentMetadata']['uri']}")

            # print(f"documentMetadata {chunk['pageSpan']['pageStart']}")
            # print(f"documentMetadata {chunk['pageSpan']['pageEnd']}")

            # print(f"previousChunks {chunk['chunkMetadata']['previousChunks'][0]['content']}")
            # print(f"nextChunks {chunk['chunkMetadata']['nextChunks'][0]['content']}")

            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            #item['pageSpan'] = f"{chunk['pageSpan']['pageStart']} ~ {chunk['pageSpan']['pageEnd']}"
            item['content'] = chunk['content']

            # chunk 는 현재 Contents에 가까운것 부터 나타남.
            p_chunks = chunk['chunkMetadata']['previousChunks']
            for p_chunk in p_chunks:
                item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

In [33]:
#question = "개인정보 보호법에 대해서 설명해주세요."

question = "국민은행 금융 상품에 대해서 알려주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)


{
  "results": [
    {
      "chunk": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/interpreter-rag-ds_1716974785800/branches/0/documents/5a452a7f20e48f6d3b3f6cf00a20e9e2/chunks/c3",
        "id": "c3",
        "content": "KB Welcome 통장.\n전자 금융/ 자동화 기기 수수료 및 환율 / 해외 송금 수수료 우대.\n\n혜택1. 전자금융/자동화기기 수수료 면제\nKB국민카드(체크카드,신용카드포함/선불카드제외) 결제실적 또는 외화 송금 보낸 실적\n있으면, 전자금융 타행이체수수료 및 KB자동화기기 시간외 출금수수료 면제\n(은행외부장소 설치된 타행제휴 기기 및 Van사기기 제외)\n\n혜택2. 환율/해외송금 수수료 우대\n환전수수료 및 해외송금수수료 50% 우대(영업점 거래시)\n\n-------\n\nKB 마이핏 통장.\n\n상품특징\n하나의 통장을 목적에 맞게 쪼개 쓸수 있는 통장 (최고 연 1.5%(세금공제전))\n가입대상\n만 18세 이상 만 38세 이하 실명의 개인 (1인 1계좌)\n※ 가입 후에는 연령과 관계 없이 KB마이핏통장 계속 이용 가능\n\n상품유형\n입출금이 자유로운 예금(저축예금)\n수수료면제\n횟수 제한 없이 아래 수수료 면제\n 다른은행 이체수수료(모바일, 인터넷, 폰뱅킹, KB국민은행 ATM)\n KB국민은행 ATM 시간 외 출금수수료\n 다른은행 ATM 출금수수료\n\n면제기간\n이번달 11일부터 다음달 10일까지\n※ 이 통장의 신규일(또는 전환일)로부터 다음 다음달 10일까지는 조건에 관계 없이 면제\n\n면제조건\n이 통장으로 전월 중 아래 실적 중 1가지 이상 충족한 경우\n KB국민카드(신용·체크·KB BC카드) 결제(현금서비스 포함) 실적\n 비상금

In [34]:
context = parse_chunks(searched_ctx)

print(context)

{'results-0': {'title': 'product', 'uri': 'gs://interpreter_rag/product.txt', 'content': 'KB국민 WELCOME PLUS 체크카드\n혜택1. 티머니(T-money) 선불카드 충전으로 대중교통 이용 가능\n\n혜택2. 대형마트(이마트,롯데마트,홈플러스) 주말 5% 할인\n건당 3만원 이상 시 할인(1회 최대 5천원 할인) ※전월 이용금액 20만원 이상 시 제공\n\n혜택3. 놀이공원(에버랜드,롯데월드,서울랜드) 30% 할인\n건당 3만원 이상 시 할인(1회 최대 15천원 할인) ※전월 이용금액 20만원 이상 시 제공\n\nKB국민 탄탄대로 웰컴 신용카드\n혜택1. 백화점(롯데,신세계,현대) 10% 청구할인\n월 최대 1만4천원 할인 ※상품권 및 온라인쇼핑몰 이용분 제외\n\n혜택2. 대중교통(버스,지하철), 택시 5% 청구할인\n월 최대 3천원 할인 ※시외, 고속버스 이용분 제외\n\n혜택3. 음식점 5% 청구할인\n월 최대 3천원 할인 ※ 용산구,영등포구,단원구 소재 음식업종 등록된 가맹점\n\n혜택4. 에버랜드 자유이용권 50%, 캐리비안베이 입장권 30% 현장할인\n※ 카드별(가족카드 포함)1일1매, 에버랜드자유이용권은 회원별 연간 10매 이내\n\n※ 계약 체결 전 상세한 혜택 및 이용조건은 홈페이지 상품설명 및 약관을 참조하시기 바랍니다.\n연체료율 연 22.9%~26.9%(고객별/기간별 차등 적용)[신용카드 남용은 가계 경제에 위협이 됩니다]\n\n-------\n\n외국인 전용 상품\n외국인 전용 상품\nKB Welcome plus 적금\n\n무료보험 가입 서비스 및 만기자금 자동 본국 송금 서비스\n\n1. 상해안심보험 무료가입서비스로 몸도 맘도 안심 PLUS\n일상생활 중 상해로 인한 사망 및 후유장해가 발생하는 경우, 최대 1천만원 보장\n(KB손해보험 플러스사랑단체보험)\n\n2. 만기자금 자동 본국송금 서비스로 어디에 있어도 안심 PLUS\n만기 시 내점이 어려운 경우를

### Gemini Pro 실행 - Vertex AI Search as a Grounding Service

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [35]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=question)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))

Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {'results-0': {'title': 'product', 'uri': 'gs://interpreter_rag/product.txt', 'content': 'KB국민 WELCOME PLUS 체크카드\n혜택1. 티머니(T-money) 선불카드 충전으로 대중교통 이용 가능\n\n혜택2. 대형마트(이마트,롯데마트,홈플러스) 주말 5% 할인\n건당 3만원 이상 시 할인(1회 최대 5천원 할인) ※전월 이용금액 20만원 이상 시 제공\n\n혜택3. 놀이공원(에버랜드,롯데월드,서울랜드) 30% 할인\n건당 3만원 이상 시 할인(1회 최대 15천원 할인) ※전월 이용금액 20만원 이상 시 제공\n\nKB국민 탄탄대로 웰컴 신용카드\n혜택1. 백화점(롯데,신세계,현대) 10% 청구할인\n월 최대 1만4천원 할인 ※상품권 및 온라인쇼핑몰 이용분 제외\n\n혜택2. 대중교통(버스,지하철), 택시 5% 청구할인\n월 최대 3천원 할인 ※시외, 고속버스 이용분 제외\n\n혜택3. 음식점 5% 청구할인\n월 최대 3천원 할인 ※ 용산구,영등포구,단원구 소재 음식업종 등록된 가맹점\n\n혜택4. 에버랜드 자유이용권 50%, 캐리비안베이 입장권 30% 현장할인\n※ 카드별(가족카드 포함)1일1매, 에버랜드자유이용권은 회원별 연간 10매 이내\n\n※ 계약 체결 전 상세한 혜택 및 이용조건은 홈페이지 상품설명 및 약관을 참조하시기 바랍니다.\n연체료율 연 22.9%~26.9%(고객별/기간별 차등 적용)[신용카드 남용은 가계 경제에 위협이 됩니다]\n\n-------\n\n외국인 전용 상품\n외국인 전용 상품\nKB Welcome plus 적금\n\n무료보험 가입 서비스 및 만기자금 자동 본국 송금 서비스\n\n1. 상해안심보험 무료가입서비스로 몸도 맘도 안심 

KB국민은행의 금융 상품은 다양합니다. 

**체크카드**

* **KB국민 WELCOME PLUS 체크카드**: 티머니 선불카드 충전으로 대중교통 이용 가능, 대형마트 주말 5% 할인, 놀이공원 30% 할인 혜택 제공
* **KB국민 WELCOME 체크카드**: 대중교통 5% 청구할인, 영화 20% 환급할인, 올리브영 10% 환급할인 혜택 제공

**신용카드**

* **KB국민 탄탄대로 웰컴 신용카드**: 백화점 10% 청구할인, 대중교통 5% 청구할인, 음식점 5% 청구할인, 에버랜드 자유이용권 50%, 캐리비안베이 입장권 30% 현장할인 혜택 제공

**적금**

* **KB Welcome plus 적금**: 외국인 전용 상품으로, 상해안심보험 무료가입 서비스, 만기자금 자동 본국 송금 서비스, WELCOME PACKAGE 우대이율 제공

**통장**

* **KB Welcome 통장**: 전자 금융/ 자동화 기기 수수료 및 환율 / 해외 송금 수수료 우대
* **KB 마이핏 통장**: 하나의 통장을 목적에 맞게 쪼개 쓸 수 있는 통장, 최고 연 1.5% 이율 제공, 다양한 수수료 면제 혜택

**주택청약종합저축**: 국민주택 및 민영주택 청약 가능, 자유적립식, 2만원 이상 50만원 이하 범위 내에서 자유납입 가능

**KB내맘대로적금**: 비대면채널 전용 DIY 상품, 정액적립식/자유적립식 선택 가능, 다양한 옵션 제공

**일반정기예금**: 목돈을 일정기간동안 예치하여 안정적인 수익을 목적으로 하는 상품

**KB국민UP정기예금**: 계단식 금리구조, 일부인출 서비스 제공, KB국민은행 및 KB국민카드 상품 거래시 우대이율 제공

**국민수퍼정기예금 (CD금리연동형)**: 매 3개월마다 CD수익률에 따라 금리가 변경 적용되는 상품

**KB Star 정기예금**: 온라인 전용 정기예금, 자동 만기관리, 분할인출 가능

**자세한 혜택 및 이용 조건은 KB국민은행 홈페이지 또는 영업점을 통해 확인하시기 바랍니다.** 
